In [238]:
import os
import sys
import torch
import numpy as np
from torch.utils.data import DataLoader
from torchmetrics.classification import MultilabelAccuracy
from torchmetrics.classification import MultilabelAUROC
import pytorch_lightning as pl
from tqdm import tqdm
import seaborn as sns
import matplotlib.pyplot as plt
import torch.nn.functional as F
sys.path.append('../..')
from multi_modal_heart.model.ecg_net_attention import ECGEncoder,ECGAttentionAE
from multi_modal_heart.model.ecg_net import ECGAE
from multi_modal_heart.model.ecg_net import BenchmarkClassifier
from multi_modal_heart.ECG.ecg_dataset import ECGDataset
from torchmetrics import Accuracy

class LitClassifier(pl.LightningModule):
    def __init__(self,encoder,input_dim,num_classes=2,lr=1e-3,freeze_encoder=False):
        super().__init__()
        self.lr =lr
        self.freeze_encoder = freeze_encoder
        self.encoder = encoder
        if self.freeze_encoder:
            self.encoder.eval()
            for param in self.encoder.parameters():
                param.requires_grad = False
        self.accu_metric = Accuracy(task="multiclass",num_classes=num_classes)
        ## initialize attention map for each class
        self.attention_map_classwise = {}
        for i in range(num_classes):
            self.attention_map_classwise[i] = []
        #### add classifier if use benchmark classifier
        self.downsteam_net = BenchmarkClassifier(input_size=input_dim,hidden_size=128,output_size=num_classes)
    def forward(self, x, mask=None):
        latent_code = self.encoder.get_features_after_pooling(x,mask)
        # print (latent_code.shape)
        return self.downsteam_net(latent_code)
    
    def training_step(self, batch, batch_idx):
        x = batch[0]
        y = batch[1]
        eid = batch[2]
        if self.freeze_encoder:
            self.encoder.eval()
            with torch.no_grad():
                latent_code = self.encoder.get_features_after_pooling(x,mask=None)
        else:
            latent_code = self.encoder.get_features_after_pooling(x,mask=None)

        y_hat = self.downsteam_net(latent_code)
        # print (y_hat.shape)
        loss = F.cross_entropy(y_hat, y)
        self.log("train_loss", loss,prog_bar=True)

        return loss
    def test_step(self, batch, batch_idx):
        x = batch[0]
        y = batch[1]
        eid_list= batch[2]
        assert len(eid_list)==len(y), "eid list and y should have the same length"
        assert x.shape[0]==len(y), "x and y should have the same length"
        logits = self(x,mask=None)
        loss = F.cross_entropy(logits, y)
        self.log("test_loss", loss)
        preds = torch.argmax(logits, dim=1)
        ## save the attention map
        lead_attention,_ = self.encoder.get_attention()
        # print(lead_attention[i].shape)
        ## make the [np.array] to numpy array
        # time_attention_map = np.array(time_attention_map)
        for i, eid in enumerate(eid_list):
            if preds[i]==y[i]:
                print(f"correct prediction: {y[i]}, eid: {eid}")
                # print(time_attention_map[:,i].shape)
                if y[i]==0:
                    self.attention_map_classwise[0].append([eid,lead_attention[i]])
                else: 
                    self.attention_map_classwise[1].append([eid,lead_attention[i]])
        self.accu_metric.update(preds, y)
        return loss
    def on_test_epoch_end(self):
        self.log("test_acc", self.accu_metric.compute())
    def validation_step(self, batch, batch_idx):
        x = batch[0]
        y = batch[1]
        eid = batch[2]
        y_hat = self(x,mask=None)
        loss = F.cross_entropy(y_hat, y)
        self.log("val_loss", loss)
        return loss
    def configure_optimizers(self):
        return torch.optim.AdamW(filter(lambda p: p.requires_grad, self.parameters()), lr=self.lr, weight_decay=1e-4)
    
def print_result(probs,super_classes_labels, topk=1):
    probs, label_indices = torch.topk(probs, topk)
    probs = probs.tolist()
    label_indices = label_indices.tolist()
    for prob, idx in zip(probs, label_indices):
        label = super_classes_labels[idx]
        print(f'{label} ({idx}):', round(prob, 4))
def calc_hamming_score(y_true, y_pred):
    return (
        (y_true & y_pred).sum(axis=1) / (y_true | y_pred).sum(axis=1)
    ).mean()    
# ecg_net = ECGAttentionAE(num_leads=12, time_steps=1024, z_dims=512, linear_out=512, downsample_factor=5, base_feature_dim=4,if_VAE=False,
#                          use_attention_pool=False,no_linear_in_E=True, apply_lead_mask=False, no_time_attention=False)
# classification_net = LitClassifier(encoder=ecg_net.encoder,input_dim=512,num_classes=5)
# # checkpoint_path  ="../../log_finetune/ECG_attention_512_raw_no_attention_pool_no_linear_abl_no_time_attention_ms_resnet/checkpoints/last-v3.ckpt"
# # checkpoint_path  ="../../log_finetune/ECG_attention_512_raw_no_attention_pool_no_linear_ms_resnet_ECG2Text/checkpoints/last-v5.ckpt"
# checkpoint_path = "../../log_finetune/ECG_attention_512_raw_no_attention_pool_no_linear_ms_resnet/checkpoints/last-v8.ckpt"
# print (torch.load(checkpoint_path)["state_dict"].keys())
# mm_checkpoint = torch.load(checkpoint_path)["state_dict"]
# encoder_params = {(".").join(key.split(".")[1:]):value for key, value in mm_checkpoint.items() if str(key).startswith("encoder")}
# classification_params = {(".").join(key.split(".")[1:]):value for key, value in mm_checkpoint.items() if str(key).startswith("downsteam_net")}
# classification_net.encoder.load_state_dict(encoder_params)
# classification_net.downsteam_net.load_state_dict(classification_params)



In [206]:
## load MI data from UKB
from scipy.stats import zscore
time_steps = 608
hf_data_path = "/home/engs2522/project/multi-modal-heart/multi_modal_heart/toolkits/ukb/non_imaging_information/HF/batched_ecg_median_wave.npy"
healthy_data_path = "/home/engs2522/project/multi-modal-heart/multi_modal_heart/toolkits/ukb/non_imaging_information/non_CVD/batched_ecg_median_wave.npy"
hf_eid_list_path  = "/home/engs2522/project/multi-modal-heart/multi_modal_heart/toolkits/ukb/non_imaging_information/HF/median_eid_list.npy"
healthy_data_eid_list_path = "/home/engs2522/project/multi-modal-heart/multi_modal_heart/toolkits/ukb/non_imaging_information/non_CVD/selected_320_healthy_subjects_eid_list.npy"

hf_data = np.load(hf_data_path)
healthy_data = np.load(healthy_data_path)
hf_data = zscore(hf_data,axis=-1)
healthy_data = zscore(healthy_data,axis=-1)
hf_data = np.nan_to_num(hf_data)
healthy_data = np.nan_to_num(healthy_data)


## pad the data to 608
pad_num = (time_steps-healthy_data.shape[-1])//2
hf_data = np.pad(hf_data,((0,0),(0,0),(pad_num,pad_num)),"constant",constant_values=0)
healthy_data = np.pad(healthy_data,((0,0),(0,0),(pad_num,pad_num)),"constant",constant_values=0)


labels = np.concatenate([np.ones(hf_data.shape[0]),np.zeros(healthy_data.shape[0])])
data = np.concatenate([hf_data,healthy_data],axis=0)
print (data.shape)
ids = np.load(hf_eid_list_path)
ids_healthy = np.load(healthy_data_eid_list_path)
ids = np.concatenate([ids,ids_healthy],axis=0)
label_eid =np.zeros((len(ids),2))
label_eid[:,0] = labels
label_eid[:,1] = ids
## split the data into train validate and test, 40% for train, 10% for validate, 50% for test
from sklearn.model_selection import train_test_split
X_trainval, X_test, y_trainval_eid, y_test_eid = train_test_split(data, label_eid, test_size=0.5, random_state=42)
X_train, X_val, y_train_eid, y_val_eid = train_test_split(X_trainval, y_trainval_eid, test_size=0.1, random_state=42)
y_test = y_test_eid[:,0]
y_train = y_train_eid[:,0]
y_val = y_val_eid[:,0]
print ('num of training data:{}, heartfailure ratio:{}'.format(X_train.shape[0],y_train.sum()))

print ('num of validation data:{}, heartfailure ratio:{}'.format(X_val.shape[0],y_val.sum()))

print ('num of test data:{}, heartfailure ratio:{}'.format(X_test.shape[0],y_test.sum()))

(638, 12, 608)
num of training data:287, heartfailure ratio:137.0
num of validation data:32, heartfailure ratio:15.0
num of test data:319, heartfailure ratio:166.0


In [207]:
import torch
import numpy as np
from torch.utils.data import TensorDataset, DataLoader
import pytorch_lightning as pl

batch_size  = 32
tensor_x = torch.Tensor(X_train) # transform to torch tensor
tensor_y = torch.Tensor(y_train).long()
tensor_eid = torch.Tensor(y_train_eid[:,1])
my_train_dataset = TensorDataset(tensor_x,tensor_y,tensor_eid) # create your datset
my_dataloader = DataLoader(my_train_dataset,batch_size=batch_size) # create your dataloader

## validation data
tensor_x = torch.Tensor(X_val) # transform to torch tensor
tensor_y = torch.Tensor(y_val).long()
tensor_eid = torch.Tensor(y_val_eid[:,1])
my_val_dataset = TensorDataset(tensor_x,tensor_y,tensor_eid) # create your datset
my_val_dataloader = DataLoader(my_val_dataset,batch_size=batch_size) # create your dataloader

## test data
tensor_x = torch.Tensor(X_test) # transform to torch tensor
tensor_y = torch.Tensor(y_test).long()
tensor_eid = torch.Tensor(y_test_eid[:,1])
my_test_dataset = TensorDataset(tensor_x,tensor_y,tensor_eid) # create your datset
my_test_dataloader = DataLoader(my_test_dataset,batch_size=batch_size) # create your dataloader

In [208]:
import torch
import sys
sys.path.append('../../')
from multi_modal_heart.model.ecg_net import ECGAE
import torch.nn as nn
import pytorch_lightning as pl
use_median_wave = True
pl.seed_everything(42)
model_name = "ECG_attention_512_raw_no_attention_pool_no_linear_ms_resnet"
if model_name=="resnet1d101_512+benchmark_classifier_ms_resnet_ECG2Text":
    ecg_net= ECGAE(encoder_type="resnet1d101",in_channels=12,ECG_length=time_steps,decoder_type="ms_resnet",
                    embedding_dim=256,latent_code_dim=512,
                    add_time=False,
                    encoder_mha = False,
                    apply_method="",
                    decoder_outdim=12)
    checkpoint_path = "/home/engs2522/project/multi-modal-heart/log_median/resnet1d101_512+benchmark_classifier_ms_resnet_ECG2Text/checkpoints/checkpoint_best_loss-v1.ckpt"
elif model_name=="resnet1d101_512+benchmark_classifier_ms_resnet":
    ecg_net= ECGAE(encoder_type="resnet1d101",in_channels=12,ECG_length=time_steps,decoder_type="ms_resnet",
                    embedding_dim=256,latent_code_dim=512,
                    add_time=False,
                    encoder_mha = False,
                    apply_method="",
                    decoder_outdim=12)
    checkpoint_path = "../../log_median/resnet1d101_512+benchmark_classifier_ms_resnet/checkpoints/checkpoint_best_loss-v1.ckpt"

elif model_name=="ECG_attention_512_raw_no_attention_pool_no_linear_ms_resnet_ECG2Text":
    ecg_net  = ECGAttentionAE(num_leads=12, time_steps=time_steps, z_dims=512, linear_out=512, 
                            downsample_factor=5, base_feature_dim=4,if_VAE=False,
                            use_attention_pool=False,
                         no_linear_in_E=True, apply_lead_mask=False, no_lead_attention=False,no_time_attention=False)
    checkpoint_path = "/home/engs2522/project/multi-modal-heart/log_median/ECG_attention_512_finetuned_no_attention_pool_no_linear_ms_resnet_ECG2Text/checkpoints/checkpoint_best_loss.ckpt"
elif model_name=="ECG_attention_512_finetuned_no_attention_pool_no_linear_ms_resnet_no_time_attention":
    ecg_net  = ECGAttentionAE(num_leads=12, time_steps=time_steps, z_dims=512, linear_out=512, 
                            downsample_factor=5, base_feature_dim=4,if_VAE=False,
                            use_attention_pool=False,
                         no_linear_in_E=True, apply_lead_mask=False, no_lead_attention=False,no_time_attention=True)
    checkpoint_path = "../../log_median/ECG_attention_512_finetuned_no_attention_pool_no_linear_ms_resnet/checkpoints/checkpoint_best_loss-v2.ckpt"
elif model_name=="ECG_attention_512_raw_no_attention_pool_no_linear_ms_resnet":
    ecg_net  = ECGAttentionAE(num_leads=12, time_steps=time_steps, z_dims=512, linear_out=512, 
                            downsample_factor=5, base_feature_dim=4,if_VAE=False,
                            use_attention_pool=False,
                         no_linear_in_E=True, apply_lead_mask=False, no_lead_attention=False,no_time_attention=False)
    checkpoint_path = "../../log_median_finetune/ECG_attention_512_raw_no_attention_pool_no_linear_ms_resnet/checkpoints/checkpoint_best_loss.ckpt"
elif model_name=="ECG_attention_512_raw_no_attention_pool_no_linear_ms_resnet_no_attention":
    ecg_net  = ECGAttentionAE(num_leads=12, time_steps=time_steps, z_dims=512, linear_out=512, 
                            downsample_factor=5, base_feature_dim=4,if_VAE=False,
                            use_attention_pool=False,
                         no_linear_in_E=True, apply_lead_mask=False, no_lead_attention=True,no_time_attention=True)
    checkpoint_path = "../../log_median/ECG_attention_512_finetuned_no_attention_pool_no_linear_ms_resnet/checkpoints/checkpoint_best_loss-v2.ckpt"
elif model_name=="ECG_attention_512_raw_no_attention_pool_no_linear_ms_resnet":
    ecg_net  = ECGAttentionAE(num_leads=12, time_steps=time_steps, z_dims=512, linear_out=512, 
                            downsample_factor=5, base_feature_dim=4,if_VAE=False,
                            use_attention_pool=False,
                         no_linear_in_E=True, apply_lead_mask=False, no_lead_attention=False,no_time_attention=False)
    checkpoint_path = "../../log_median/ECG_attention_512_finetuned_no_attention_pool_no_linear_ms_resnet/checkpoints/checkpoint_best_loss.ckpt"
else:
    raise NotImplementedError

num_classes = 2 ## for binary classification
freeze_encoder = True
train_from_scratch = False
# resnet_checkpoint = '../../log_finetune/resnet1d101_512+benchmark_classifier_ms_resnet/checkpoints/epoch=23-val_auroc:benchmark_classifier/val_macro_auc=0.91.ckpt'
# checkpoint = torch.load(checkpoint_path)["state_dict"]
# encoder_params = {(".").join(key.split(".")[2:]):value for key, value in checkpoint.items() if str(key).startswith("network.encoder")}
# classification_params = {(".").join(key.split(".")[2:]):value for key, value in checkpoint.items() if str(key).startswith("network.downsteam_net")}
if not train_from_scratch:
    print("load checkpoint from {}".format(checkpoint_path))
    checkpoint = torch.load(checkpoint_path)["state_dict"]
    classification_net = LitClassifier(encoder=ecg_net.encoder,input_dim=512,num_classes=num_classes,lr=1e-3,freeze_encoder=freeze_encoder)

    try:
        encoder_params = {(".").join(key.split(".")[2:]):value for key, value in checkpoint.items() if str(key).startswith("network.encoder")}
        classification_net.encoder.load_state_dict(encoder_params)
    except:
        encoder_params = {(".").join(key.split(".")[1:]):value for key, value in checkpoint.items() if str(key).startswith("encoder")}
        classification_net.encoder.load_state_dict(encoder_params)
    max_epochs = 50
    if freeze_encoder:
        classification_net.encoder.eval()
        for param in classification_net.encoder.parameters():
            param.requires_grad = False
else:
    classification_net = LitClassifier(encoder=ecg_net.encoder,input_dim=512,num_classes=num_classes,lr=1e-3,freeze_encoder=freeze_encoder)
    max_epochs = 400


Global seed set to 42


no linear layer
load checkpoint from ../../log_median_finetune/ECG_attention_512_raw_no_attention_pool_no_linear_ms_resnet/checkpoints/checkpoint_best_loss.ckpt


In [209]:
import torch
import sys
import pytorch_lightning as pl
from pytorch_lightning.loggers import TensorBoardLogger
import os
tb_logger = TensorBoardLogger( f"./finetune_on_HF", name=model_name, version="")  
checkpoint_dir  = os.path.join(tb_logger.log_dir,"checkpoints")

checkpoint_callback_best_loss_min = pl.callbacks.ModelCheckpoint(dirpath=checkpoint_dir, 
                                                    filename='checkpoint_best_loss',
                                                    save_top_k=1, monitor="val_loss"
                                                    , mode='min',save_last=True)


callbacks=[
    # FineTuneLearningRateFinder(milestones=[5, 10],min_lr=1e-5, max_lr=1e-3, 
    #                             mode='linear', early_stop_threshold=4.0),
    checkpoint_callback_best_loss_min
    ]


trainer = pl.Trainer(accelerator="gpu",
                    devices=1, max_epochs=max_epochs,
                    logger=tb_logger,log_every_n_steps=1,check_val_every_n_epoch = 1,
                    callbacks=callbacks,
                    )
trainer.fit(classification_net, train_dataloaders=my_dataloader,val_dataloaders=my_val_dataloader)


GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0,1]

  | Name          | Type                | Params
------------------------------------------------------
0 | encoder       | ECGEncoder          | 1.3 M 
1 | accu_metric   | MulticlassAccuracy  | 0     
2 | downsteam_net | BenchmarkClassifier | 67.2 K
------------------------------------------------------
67.2 K    Trainable params
1.3 M     Non-trainable params
1.3 M     Total params
5.367     Total estimated model params size (MB)


Sanity Checking: 0it [00:00, ?it/s]

Training: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

...

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

`Trainer.fit` stopped: `max_epochs=50` reached.


In [210]:
print(len(y_test))

319


In [211]:
result = trainer.test(classification_net,my_test_dataloader,ckpt_path="best")

## save the model
torch.save(classification_net.state_dict(),f"./finetune_on_HF/{model_name}/checkpoints/final.pt")

Restoring states from the checkpoint path at /home/engs2522/project/multi-modal-heart/multi_modal_heart/tasks/finetune_on_HF/ECG_attention_512_raw_no_attention_pool_no_linear_ms_resnet/checkpoints/checkpoint_best_loss-v33.ckpt
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0,1]
Loaded model weights from the checkpoint at /home/engs2522/project/multi-modal-heart/multi_modal_heart/tasks/finetune_on_HF/ECG_attention_512_raw_no_attention_pool_no_linear_ms_resnet/checkpoints/checkpoint_best_loss-v33.ckpt


Testing: 0it [00:00, ?it/s]

correct prediction: 1, eid: 5349144.0
correct prediction: 1, eid: 4719122.0
correct prediction: 1, eid: 1588791.0
correct prediction: 0, eid: 2686942.0
correct prediction: 0, eid: 3210667.0
correct prediction: 0, eid: 1904246.0
correct prediction: 1, eid: 2710857.0
correct prediction: 0, eid: 1529499.0
correct prediction: 0, eid: 1479281.0
correct prediction: 0, eid: 1814059.0
correct prediction: 0, eid: 2933217.0
correct prediction: 0, eid: 3301738.0
correct prediction: 1, eid: 4493644.0
correct prediction: 0, eid: 3817912.0
correct prediction: 1, eid: 1949122.0
correct prediction: 1, eid: 1066856.0
correct prediction: 0, eid: 4674759.0
correct prediction: 0, eid: 2154279.0
correct prediction: 1, eid: 1992944.0
correct prediction: 1, eid: 5621490.0
correct prediction: 0, eid: 5682669.0
correct prediction: 1, eid: 2544886.0
correct prediction: 0, eid: 2547362.0
correct prediction: 1, eid: 5596059.0
correct prediction: 1, eid: 3689767.0
correct prediction: 1, eid: 2463161.0
correct pred

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃        Test metric        ┃       DataLoader 0        ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│         test_acc          │    0.7304075360298157     │
│         test_loss         │    0.6435382962226868     │
└───────────────────────────┴───────────────────────────┘

In [214]:
## load the model
classification_net.load_state_dict(torch.load("./finetune_on_HF/ECG_attention_512_raw_no_attention_pool_no_linear_ms_resnet/checkpoints/final.pt"))

<All keys matched successfully>

In [260]:
## get attention map
attention_map_classwise = classification_net.attention_map_classwise
## norm group
print (len(attention_map_classwise))
norm_group_attention = attention_map_classwise[0]
hf_group_attention = attention_map_classwise[1]
print (len(norm_group_attention))
print (len(hf_group_attention))
norm_attention_maps= [v[1] for v in norm_group_attention]
hf_attention_maps = [v[1] for v in hf_group_attention]

norm_attention_maps = np.stack(norm_attention_maps,axis=0)
hf_attention_maps = np.stack(hf_attention_maps,axis=0)

average_attention_map_norm = np.mean(norm_attention_maps,axis=0)
average_attention_map_hf = np.mean(hf_attention_maps,axis=0)
## std variance
var_attention_map_norm = np.std(norm_attention_maps,axis=0)
var_attention_map_hf = np.std(hf_attention_maps,axis=0)

print(average_attention_map_hf.shape)

2
108
125
(12, 12)


In [263]:
import seaborn as sns
import matplotlib.pyplot as plt
sns.set(font_scale=1.5)
x_ticks = ["I","II","III","aVR","aVL","aVF","V1","V2","V3","V4","V5","V6"]
# cmap  =sns.palplot(sns.color_palette("Blues",12))
colormap = sns.color_palette("Greens")
fig,axes = plt.subplots(ncols=2,nrows=2,figsize=(20,20),dpi=500)

sns.heatmap(average_attention_map_norm,cmap=colormap,fmt=".2f",annot=False,xticklabels=x_ticks,yticklabels=x_ticks,ax=axes[0,0])
sns.heatmap(average_attention_map_hf,cmap=colormap,fmt=".2f",annot=False,xticklabels=x_ticks,yticklabels=x_ticks,ax=axes[0,1])

sns.heatmap(var_attention_map_norm,cmap=colormap,fmt=".2f",annot=False,xticklabels=x_ticks,yticklabels=x_ticks,ax=axes[1,0])
sns.heatmap(var_attention_map_hf,cmap=colormap,fmt=".2f",annot=False,xticklabels=x_ticks,yticklabels=x_ticks,ax=axes[1,1])

axes[0,0].set_title("control group [AVG]")
axes[0,1].set_title("heart failure group [AVG]")
axes[1,0].set_title("control group [STD]")
axes[1,1].set_title("heart failure group [STD]")

## change xticks
# plt.xticks(np.arange(0,12),x_ticks,rotation=45)
# plt.xlabel("leads")
# ## change yticks
# plt.yticks(np.arange(0,12),x_ticks,rotation=45)
## put x ticks on the top
# ax = plt.gca()
# ax.xaxis.tick_top()

Text(0.5, 1.0, 'heart failure group [STD]')

: 

[[tensor(2686942., device='cuda:0'),
  tensor([[8.7061e-23, 9.7653e-11, 1.7466e-08, 1.0103e-13, 1.2500e-01, 2.5000e-01,
           1.8980e-30, 3.7500e-01, 6.9558e-14, 1.2500e-01, 1.1497e-14, 1.2500e-01],
          [6.1322e-23, 2.2624e-02, 1.2500e-01, 9.5935e-10, 1.2500e-01, 2.2734e-01,
           6.6657e-11, 1.2504e-01, 1.6755e-07, 2.5000e-01, 6.6888e-10, 1.2500e-01],
          [3.5108e-30, 6.9553e-03, 2.1245e-18, 3.6830e-17, 1.2500e-01, 2.4304e-01,
           1.5414e-44, 3.7500e-01, 2.4982e-14, 1.2500e-01, 2.5140e-28, 1.2500e-01],
          [6.8131e-38, 1.3173e-05, 5.1690e-02, 3.2685e-07, 1.2500e-01, 2.4999e-01,
           2.8351e-17, 2.8034e-01, 4.2972e-02, 1.2500e-01, 6.6244e-20, 1.2500e-01],
          [4.6239e-29, 2.7987e-02, 4.2737e-17, 6.7053e-16, 1.2500e-01, 2.2201e-01,
           2.8126e-39, 3.7500e-01, 6.3991e-13, 1.2500e-01, 9.5584e-22, 1.2500e-01],
          [1.3540e-26, 2.4580e-01, 1.2491e-01, 1.9993e-07, 1.2500e-01, 1.2602e-01,
           1.3963e-10, 2.5318e-01, 9.2504e-05